In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from jppype import Mosaic, vscode_theme

from fundus_vessels_toolkit.pipelines import FundusVesselSegToGraph
from fundus_vessels_toolkit.segment import clear_gpu_cache, segment_vessels
from fundus_vessels_toolkit.segment_to_graph import populate_geometry
from fundus_vessels_toolkit.utils.data_io import load_image

vscode_theme()

HTML(value="<style>\n        .cell-output-ipywidget-background {\n                background: transparent !imp…

#### Load image

In [3]:
# Path to the raw fundus image
RAW_PATH = "/home/gaby/These/Data/Fundus/Vessels/FIVES/test/downsampled-1024/1-images/103_G.jpg"

# Load the raw image
raw = load_image(RAW_PATH)

# Segment the vessels
vessels = segment_vessels(raw)

# Clear the GPU cache
clear_gpu_cache()

#### Parse the vascular graph AND its geometry

In [4]:
max_vessel_diameter = raw.shape[1] // 50
seg2graph = FundusVesselSegToGraph(max_vessel_diameter, parse_geometry=True)
# seg2graph.simplify_topology = None
graph = seg2graph(vessels)

#### Display the graph

In [5]:
mosaic = Mosaic(3, raw, cell_height=800, cols_titles=["Topology", "Cleaned Skeleton", "BSplines"])
mosaic.add_label(vessels, "vessel", opacity=0.3)

mosaic[0]["vgraph"] = graph.jppype_layer(node_labels=True)
mosaic[1]["vgraph"] = graph.jppype_layer(edge_map=True, edge_labels=True, boundaries=True)
mosaic[2]["vgraph"] = graph.jppype_layer(bspline=True, edge_labels=True)
mosaic[2]["bsplines_nodes"] = graph.geometric_data().jppype_bspline_tangents(scaling=0.4)

mosaic.show()

GridBox(children=(HTML(value='<h3 style="text-align: center;">Topology</h3>'), HTML(value='<h3 style="text-ali…

In [6]:
from fundus_vessels_toolkit.segment_to_graph.graph_simplification import reconnect_endpoints

graph2 = reconnect_endpoints(graph, intercept_snapping_distance=20, inplace=False)

In [7]:
from jppype.utils.color import colormap_by_name

mosaic = Mosaic(2, raw, cell_height=800)
mosaic.add_label(vessels, "vessel", opacity=0.3)

mosaic[0]["vgraph"] = graph.jppype_layer(bspline=True, edge_labels=True, node_labels=True)
mosaic[0]["tangent"] = graph.geometric_data().jppype_bspline_tangents(scaling=1)
mosaic[0]["vgraph"].nodes_cmap = "grey"
mosaic[0]["vgraph"].edges_cmap = "grey"
mosaic[1]["vgraph"] = graph2.jppype_layer(bspline=True, edge_labels=True, node_labels=True)
mosaic[1]["vgraph"].nodes_cmap = {k: "grey" for k in range(graph.nodes_count)} | {None: colormap_by_name()}
mosaic[1]["vgraph"].edges_cmap = {k: "grey" for k in range(graph.branches_count)} | {None: colormap_by_name()}

mosaic.show()

GridBox(children=(View2D(linkedTransformGroup='a3395ff1d17540cb9b50b508607fe5b0'), View2D(linkedTransformGroup…

In [8]:
graph2.branch_list[70]

array([74,  3])

### Access geometrical data

The geometrical representations of vascular graphs are stored in ``VGeometricData`` objects. Usually ``VGraph``  are linked to a single ``VGeometricData`` object, but actually it may be linked to more than one when several geometrical versions of the same graph are available (e.g. different image of the same patient registered together). In any case, the geometrical data can be accessed through the ``VGraph`` objects by calling the ``geometric_data()`` method (with an optional id).

``VGeometricData`` object are only containers and only provide method to access the geometrical data, they also ensure that the data stay coherent when the graph is modified (e.g. when calling``graph.delete_nodes()``, ``VGraph.reindex_branches()``, ``VGraph.merge_nodes()``, ``VGraph.split_branches()``, etc). 

However they don't provide tools to generate the geometrical data. This is done by the ``fundus_vessels_toolkit.segment_to_graph.geometry_parsing.populate_geometry()`` method or by enabling the ``parse_geometry`` option in the ``FundusVesselSegToGraph`` pipelines.

In [9]:
geo_data = graph.geometric_data()
list(geo_data.branch_data().keys())

['TIPS_TANGENT',
 'TIPS_CALIBRE',
 'TIPS_BOUNDARIES',
 'TANGENTS',
 'BSPLINE',
 'CALIBRES',
 'BOUNDARIES',
 'CURVATURES',
 'CURVATURE_ROOTS']

#### Extract bifurcation infos

In [10]:
from fundus_vessels_toolkit import VBranchGeoData
from fundus_vessels_toolkit.utils.graph.measures import extract_bifurcations_parameters

# Access the calibre and tangent data for each branch
# Those metrics were previously computed when calling ``seg2graph(vessels)`` with parse_geometry=True
geo_data = graph.geometric_data()
calibres = geo_data.branch_data(VBranchGeoData.Fields.CALIBRES)
tangents = geo_data.branch_data(VBranchGeoData.Fields.TANGENTS)
extract_bifurcations_parameters(calibres, tangents, graph.branch_list, False)

,b0,b1,b2,θ1,θ2,d0,d1,d2
nodeID,,,,,,,,
3,54,30,31,14.437885,60.251446,11.284188,10.105234,2.151480
4,18,17,19,10.340105,46.482489,16.779570,12.070123,10.967249
5,34,33,0,0.000008,65.182264,13.000000,10.500002,8.002392
6,54,55,8,12.054130,81.796130,11.284188,10.915263,3.720051
9,56,55,29,-0.000084,67.900005,11.923350,10.100000,3.954156
10,9,7,8,42.838569,72.994939,13.029297,12.043381,4.827565
12,64,5,65,9.175730,65.582299,15.149419,12.223966,7.753165
27,19,24,25,13.621239,26.314922,12.998276,7.232947,5.267303
30,44,46,45,39.069409,58.434038,8.823377,6.624612,3.415363
